In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
import random
import torch
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold
import tokenizers
from transformers import RobertaModel, RobertaConfig

from tqdm import tqdm

In [ ]:
train_df = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
train_df['text'] = train_df['text'].astype(str)
train_df['selected_text'] = train_df['selected_text'].astype(str)
t = {}
for i in range(len(train_df)):
    if train_df['sentiment'][i] == 'neutral': continue
    w = train_df['selected_text'][i][0]
    if w == '#':
        print('---------')
        print(train_df['text'][i])
        print(train_df['selected_text'][i])
        
    if not w.isalpha():
        if w not in t:
            t[w] = 1
        else:
            t[w] += 1
t

In [ ]:
def check(a, b, l):
    for i in range(len(b)):
        if b[i] == ' ' and a[l] != ' ': 
            continue 
        if b[i] != ' ' and l <= len(a)- 1 and a[l] == ' ':
            l += 1
            continue
        if l <= len(a) - 1 and b[i] == a[l]:
            l += 1
        else:
            return False          
    return True
        

In [ ]:
# base = pd.read_csv('../input/baseline-result/baseline_result.csv')
# test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')
# print(len(base))
# num = 0
# t1 = 0
# t2 = 0
# t3 = 0
# t4 = 0
# tot = 0
# for i in range(len(base)):
#     a = test['text'][i].lower()
#     b = base['selected_text'][i].lower()
#     while b[0] == ' ':
#         b = b[1:]
#     for j in range(len(a)):
#         if check(a, b, j):
#             if j - 1 >=0 and a[j-1] != ' ' and not a[j-1].isalpha():
#                 num += 1
#                 print('------------')
#                 print(a)
#                 print(b)
#             if j - 2 >= 0 and a[j-1] == ' ' and  a[j-2] != ' ' and not a[j-2].isalpha():
#                 num += 1
#                 print(a)
#                 print(b)
#                 print('------------')
#             if j - 3 >=0 and a[j-1] == ' ' and a[j-2] == ' ' and a[j-3] != ' ' and not a[j-3].isalpha():
#                 print(a)
#                 print(b)
#                 print('---------')
#                 num += 1
#             if j - 4 >=0 and a[j-1] == ' ' and a[j-2] == ' '  and a[j-3] == ' ' and not a[j-4].isalpha():
#                 print(a)
#                 print(b)
#                 print('---------')
#                 num += 1
#             break
# print(num)


In [ ]:
# base = pd.read_csv('../input/baseline-result/baseline_result.csv')
# test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')
# for i in range(len(base)):
#     a = test['text'][i].lower()
#     b = base['selected_text'][i].lower()
#     while b[0] == ' ':
#         b = b[1:]
#     for j in range(len(a)):
#         if check(a, b, j):
#             if j - 1 >=0 and a[j-1] != ' ' and not a[j-1].isalpha():
#                 b = a[j-1] + ' ' + b
#             if j - 2 >= 0 and a[j-1] == ' ' and  a[j-2] != ' ' and not a[j-2].isalpha():
#                 b = a[j-2] + ' ' + b
#             if j - 3 >=0 and a[j-1] == ' ' and a[j-2] == ' ' and a[j-3] != ' ' and not a[j-3].isalpha():
#                 b = a[j-3] + ' ' + b
#             if j - 4 >=0 and a[j-1] == ' ' and a[j-2] == ' '  and a[j-3] == ' ' and not a[j-4].isalpha():
#                 b = a[j-4] + ' ' + b
#             break


In [ ]:
# def pp(text, str2):

#     text = text.lower()
#     str2 = str2.lower()
#     print('-------')
#     print(str2)
#     print(str2[0])
#     while str2[0] == ' ':
#         str2 = str2[1:]
#     for j in range(len(text)):
#         if check(text, str2, j):
#             if j - 1 >=0 and text[j-1] != ' ' and not text[j-1].isalpha():
#                 str2 = text[j-1] + ' ' + str2
#             if j - 2 >= 0 and text[j-1] == ' ' and  text[j-2] != ' ' and not text[j-2].isalpha():
#                 str2 = text[j-2] + ' ' + str2
#             if j - 3 >=0 and text[j-1] == ' ' and text[j-2] == ' ' and text[j-3] != ' ' and not text[j-3].isalpha():
#                 str2 = text[j-3] + ' ' + str2
#             if j - 4 >=0 and text[j-1] == ' ' and text[j-2] == ' '  and text[j-3] == ' ' and text[j-4] != ' ' and not text[j-4].isalpha():
#                 str2 = text[j-4] + ' ' + str2
#             break
#     return str2

In [ ]:
# pre = pd.read_csv('../input/pre-selected-text/pre.csv')
# print(len(pre))
# text = []
# selected_text = []
# p = []
# sentiment = []
# for i in range(len(pre)):
#     if pre['sentiment'][i] == 'neutral': continue
#     if pre['selected_text'][i].lower().split() != pre['pre_selected_text'][i].lower().split() and not pre['selected_text'][i][0].isalpha():
        
#         text.append(pre['text'][i])
#         selected_text.append(pre['selected_text'][i])
#         p.append(pre['pre_selected_text'][i])
#         sentiment.append(pre['sentiment'][i])

# d = {'text': pd.Series(text), 
#      'selected_text': pd.Series(selected_text),
#      'pre_selected_text': pd.Series(p),
#      'sentiment': pd.Series(sentiment)}
# df = pd.DataFrame(d)
# t1 = 0
# t2 = 0
# t3 = 0
# for i in range(len(df)):
#     if '!' in df['text'][i]:
#         t1 += 1
#     if '!' in df['selected_text'][i]:
#         t2 += 1
#     if '!' in df['pre_selected_text'][i]:
#         t3 += 1
# print(t1, t2, t3)
# print(len(df))
# df.to_csv('an.csv')
# print(len(selected_text))
# print(len(p))
# print(len(sentiment))
        

In [ ]:
# train_df = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
# train_df['text'] = train_df['text'].astype(str)
# train_df['selected_text'] = train_df['selected_text'].astype(str)
# train_neutral = train_df[train_df['sentiment'] == 'neutral']
# train_neutral.to_csv('train_neutral.csv')
# train_positive = train_df[train_df['sentiment'] == 'positive']
# train_positive.to_csv('train_positive.csv')
# train_negative = train_df[train_df['sentiment'] == 'negative']
# train_negative.to_csv('train_negative.csv')
# print(len(train_neutral), len(train_positive), len(train_negative))


#  **Seed**

In [ ]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed = 42
seed_everything(seed)

# **Data Loader**

In [ ]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, df, max_len=96):
        self.df = df
        self.max_len = max_len
        self.labeled = 'selected_text' in df
        self.tokenizer = tokenizers.ByteLevelBPETokenizer(
            vocab_file='../input/roberta/vocab.json', 
            merges_file='../input/roberta/merges.txt', 
            lowercase=True,
            add_prefix_space=True)

    def __getitem__(self, index):
        data = {}
        row = self.df.iloc[index]
        
        ids, masks, tweet, offsets = self.get_input_data(row)
        data['ids'] = ids
        data['masks'] = masks
        data['tweet'] = tweet
        data['offsets'] = offsets
        data['sentiment'] = row.sentiment
      
        data['selected_text'] = row.text
        if self.labeled:
            start_idx, end_idx = self.get_target_idx(row, tweet, offsets)
            data['start_idx'] = start_idx
            data['end_idx'] = end_idx
        
        return data

    def __len__(self):
        return len(self.df)
    
    def get_input_data(self, row):
        
        tweet = " " + " ".join(row.text.lower().split())
        
        encoding = self.tokenizer.encode(tweet)
       
        sentiment_id = self.tokenizer.encode(row.sentiment).ids
        
        ids = [0] + sentiment_id + [2, 2] + encoding.ids + [2]
        
        offsets = [(0, 0)] * 4 + encoding.offsets + [(0, 0)]
        
        pad_len = self.max_len - len(ids)
        if pad_len > 0:
            ids += [1] * pad_len
            offsets += [(0, 0)] * pad_len
        
        ids = torch.tensor(ids)
        masks = torch.where(ids != 1, torch.tensor(1), torch.tensor(0))
        offsets = torch.tensor(offsets)
        
        return ids, masks, tweet, offsets
        
    def get_target_idx(self, row, tweet, offsets):
        selected_text = " " +  " ".join(row.selected_text.lower().split())

        len_st = len(selected_text) - 1
        idx0 = None
        idx1 = None

        for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
            if " " + tweet[ind: ind+len_st] == selected_text:
                idx0 = ind
                idx1 = ind + len_st - 1
                break

        char_targets = [0] * len(tweet)
        if idx0 != None and idx1 != None:
            for ct in range(idx0, idx1 + 1):
                char_targets[ct] = 1

        target_idx = []
        for j, (offset1, offset2) in enumerate(offsets):
            if sum(char_targets[offset1: offset2]) > 0:
                target_idx.append(j)

        start_idx = target_idx[0]
        end_idx = target_idx[-1]
        
        return start_idx, end_idx
        
def get_train_val_loaders(df, train_idx, val_idx, batch_size=8):
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]

    train_loader = torch.utils.data.DataLoader(
        TweetDataset(train_df), 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=2,
        drop_last=True)

    val_loader = torch.utils.data.DataLoader(
        TweetDataset(val_df), 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2)

    dataloaders_dict = {"train": train_loader, "val": val_loader}

    return dataloaders_dict

def get_test_loader(df, batch_size=32):
    loader = torch.utils.data.DataLoader(
        TweetDataset(df), 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2)    
    return loader

# **Model**

In [ ]:
class Build_model(nn.Module):
    def __init__(self):
        super(Build_model, self).__init__()
        config = RobertaConfig.from_pretrained('../input/roberta/config.json', output_hidden_states=True)
        self.roberta = RobertaModel.from_pretrained('../input/roberta/pytorch_model.bin', config=config)
        self.dropout = nn.Dropout(0.5)
        self.final_projection = nn.Linear(config.hidden_size, 2)
        nn.init.normal_(self.final_projection.weight, std=0.02)
        nn.init.normal_(self.final_projection.bias, 0)
        
        self.w1 = torch.nn.Parameter(torch.FloatTensor(1), requires_grad=True)
        self.w2 = torch.nn.Parameter(torch.FloatTensor(1), requires_grad=True)
        self.w3 = torch.nn.Parameter(torch.FloatTensor(1), requires_grad=True)
        self.w1.data.fill_(0.3)
        self.w2.data.fill_(0.3)
        self.w3.data.fill_(0.3)
    
    def forward(self, input_ids, attention_mask):
#         self.w1 = nn.ReLU(self.w1)
#         self.w2 = nn.ReLU(self.w2)
#         self.w3 = nn.ReLU(self.w3)
#         w = self.w1 + self.w2 + self.w3
        _, _, hs = self.roberta(input_ids, attention_mask)
        x = torch.stack([hs[-1], hs[-2], hs[-3]])
        x = torch.mean(x, 0)
#         x = (self.w1 / w) * hs[-1] + (self.w2 / w) * hs[-1] + (self.w3 / w) * hs[-3]
        x = self.dropout(x)
        x = self.final_projection(x)
        start_prediction, end_prediction = x.split(1, dim=-1)
        start_prediction = start_prediction.squeeze(-1)
        end_prediction = end_prediction.squeeze(-1)
        return start_prediction, end_prediction

# **Loss Function**

In [ ]:
def loss_function(start_prediction, end_prediction, start_target, end_target):
    nn_cross = nn.CrossEntropyLoss()
    start_loss = nn_cross(start_prediction, start_target)
    end_loss = nn_cross(end_prediction, end_target)
    return start_loss + end_loss

# **Evaluation Function**

In [ ]:
def pp(text, str2):
    if str2 == '': return str2
    text = text.lower()
    str2 = str2.lower()

    while str2[0] == ' ':
        str2 = str2[1:]
    for j in range(len(text)):
        if check(text, str2, j):
            if j - 1 >=0 and text[j-1] != ' ' and not text[j-1].isalpha():
                str2 = text[j-1] + ' ' + str2
                return str2
            if j - 2 >= 0 and text[j-1] == ' ' and  text[j-2] != ' ' and not text[j-2].isalpha():
                str2 = text[j-2] + ' ' + str2
                return str2
#             if j - 3 >=0 and text[j-1] == ' ' and text[j-2] == ' ' and text[j-3] != ' ' and not text[j-3].isalpha():
#                 str2 = text[j-3] + ' ' + str2
#                 return str2
#             if j - 4 >=0 and text[j-1] == ' ' and text[j-2] == ' '  and text[j-3] == ' ' and text[j-4] != ' ' and not text[j-4].isalpha():
#                 str2 = text[j-4] + ' ' + str2
#                 return str2
            break
    return str2

In [ ]:
def get_selected_text(text, start_idx, end_idx, offsets):
    selected_text = ""
    for ix in range(start_idx, end_idx + 1):
        selected_text += text[offsets[ix][0]: offsets[ix][1]]
        if (ix + 1) < len(offsets) and offsets[ix][1] < offsets[ix + 1][0]:
            selected_text += " "
    return selected_text

def jaccard(str1, str2, text): 
    
#     aa = str1
#     bb = str2
#     a = set(str1.lower().split()) 
#     b = set(str2.lower().split())
#     c = a.intersection(b)
#     s1 = float(len(c)) / (len(a) + len(b) - len(c))
    
    if len(str2.split()) == 1:
        str2 = pp(text, str2)
    str2 = pp(text, str2)
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    s2 = float(len(c)) / (len(a) + len(b) - len(c))
    return s2
#     if len(str2) >= 1 and str2[0] == ',' and len(str2)<=5:
#         if s1 > s2: 
#             print('bad')
#             print(text)
#             print(aa)
#             print(bb)
#             print(str2)
#             print(s1, s2)
#         if s1 < s2: 
#             print('good')
#             print(s1, s2)
# #         print('--------')
# #         print(text)
# #         print(aa)
# #         print(bb)
# #         print(str2)
#         return s2
#     return s1

#     if s1 > s2:
#         print('-------------')
#         print(text)
#         print(aa)
#         print(bb)
#         print(str2)
#     if s1 > s2:
        
#         print('-------------')
#         print(len(bb))
#         print(text)
#         print(aa)
#         print(bb)
#         print(str2)
#    return max(s1, s2)

def compute_jaccard_score(text, start_idx, end_idx, start_logits, end_logits, offsets):
    start_pred = np.argmax(start_logits)
    end_pred = np.argmax(end_logits)
    if start_pred > end_pred:
        pred = text
    else:
        pred = get_selected_text(text, start_pred, end_pred, offsets)
        
    true = get_selected_text(text, start_idx, end_idx, offsets)
    
    return jaccard(true, pred, text)

In [ ]:
# jaccard(',       rather', ', rather tiredd')

# **Training & Prediction**

In [ ]:
def train_or_eval_model(model, dataloaders_dict, criterion, optimizer, num_epochs, filename):
    model.cuda()
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            epoch_loss = 0.0
            epoch_jaccard = 0.0
            #progress_bar = tqdm(dataloaders_dict[phase], desc='| Epoch {:03d}'.format(epoch), leave=False, disable=False)
            #for i, data in enumerate(progress_bar):
            for data in (dataloaders_dict[phase]):
                ids = data['ids'].cuda()
                masks = data['masks'].cuda()
                tweet = data['tweet']
                offsets = data['offsets'].numpy()
                start_idx = data['start_idx'].cuda()
                end_idx = data['end_idx'].cuda()
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):

                    start_logits, end_logits = model(ids, masks)
                    loss = criterion(start_logits, end_logits, start_idx, end_idx)
                
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * len(ids)
                    
                    start_idx = start_idx.cpu().detach().numpy()
                    end_idx = end_idx.cpu().detach().numpy()
                    start_logits = torch.softmax(start_logits, dim=1).cpu().detach().numpy()
                    end_logits = torch.softmax(end_logits, dim=1).cpu().detach().numpy()
                    
                    for i in range(len(ids)):                        
                        jaccard_score = compute_jaccard_score(
                            tweet[i],
                            start_idx[i],
                            end_idx[i],
                            start_logits[i], 
                            end_logits[i], 
                            offsets[i])
                        epoch_jaccard += jaccard_score
                    #progress_bar.set_postfix({'loss': epoch_loss / len(dataloaders_dict[phase].dataset), 'jac': epoch_jaccard/len(dataloaders_dict[phase].dataset)}, refresh=False)
                    
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_jaccard = epoch_jaccard / len(dataloaders_dict[phase].dataset)
            print('Epoch {}/{} | {:^5} | Loss: {:.4f} | Jaccard: {:.4f}'.format(
                epoch + 1, num_epochs, phase, epoch_loss, epoch_jaccard))
    
    torch.save(model.state_dict(), filename)

# CHECK****

In [ ]:
# k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
# test = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
# test['text'] = test['text'].astype(str)
# test['selected_text'] = test['selected_text'].astype(str)
# test_loader = get_test_loader(test)
# predictions = []
# models = []
# for fold in range(k_fold.n_splits):
#     model = Build_model()
#     model.cuda()
#     file_name = f'pseudo_fold{fold+1}.pth'
#     file_name = '../input/parameter/' + file_name
#     model.load_state_dict(torch.load(file_name))
#     model.eval()
#     models.append(model)
# num = 0
# for data in test_loader:
#     ids = data['ids'].cuda()
#     masks = data['masks'].cuda()
#     tweet = data['tweet']
#     offsets = data['offsets'].numpy()
#     sentiment = data['sentiment']
#     selected_text = data['selected_text']

#     start_logits = []
#     end_logits = []
#     for model in models:
#         with torch.no_grad():
#             output = model(ids, masks)
#             start_logits.append(torch.softmax(output[0], dim=1).cpu().detach().numpy())
#             end_logits.append(torch.softmax(output[1], dim=1).cpu().detach().numpy())

#     start_logits = np.mean(start_logits, axis=0)
#     end_logits = np.mean(end_logits, axis=0)
    
#     for i in range(len(ids)): 
#         if sentiment[i] != 'neutral':
#             start_pred = np.argmax(start_logits[i])
#             end_pred = np.argmax(end_logits[i])
#             if start_pred > end_pred:
#                 pred = tweet[i]
#             else:
#                 pred = get_selected_text(tweet[i], start_pred, end_pred, offsets[i])
#             predictions.append(pred)
#         else:
#             num += 1
#             predictions.append(selected_text[i])

In [ ]:
# print(len(predictions))

In [ ]:
# test['pre_selected_text'] = predictions
# test.to_csv('pre.csv')

In [ ]:
# text = []
# selected_text = []
# pre_selected_text = []
# sentiment = []
# for i in range(len(test)):


# **Main**
**I.**

In [ ]:

# epochs = 3
# batch_size = 32
# k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# train_df = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
# train_df['text'] = train_df['text'].astype(str)
# train_df['selected_text'] = train_df['selected_text'].astype(str)

# #train_neutral = train_df[train_df['sentiment'] == 'neutral']
# #train_neutral.to_csv('train_neutral.csv')
# #train_positive = train_df[train_df['sentiment'] == 'positive']
# #train_positive.to_csv('train_positive.csv')
# #train_negative = train_df[ train_df['sentiment'] == 'negative']
# #train_negative.to_csv('train_negative.csv')
# train_2 = train_df[train_df['sentiment'] != 'neutral']
# print(len(train_2))
# for fold,(train_idx, val_idx) in enumerate(k_fold.split(train_2, train_2.sentiment), start=1):
#     print(f'Fold: {fold}')
#     model = Build_model()
#     optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, betas=(0.9, 0.999))
#     criterion = loss_function
#     dataloaders_dict = get_train_val_loaders(train_2, train_idx, val_idx,batch_size)
    
#     train_or_eval_model(model, dataloaders_dict, criterion, optimizer, epochs, f'roberta_fold{fold}.pth')
    

II.

In [ ]:
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
test_df = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')
test_df['text'] = test_df['text'].astype(str)
test_loader = get_test_loader(test_df)
predictions = []
models = []
for fold in range(k_fold.n_splits):
    print(fold)
    model = Build_model()
    model.cuda()
    #model.load_state_dict(torch.load(f'roberta_fold{fold+1}.pth'))
    file_name = f'pseudo_fold{fold+1}.pth'
    file_name = '../input/parameter/' + file_name
    model.load_state_dict(torch.load(file_name))
    model.eval()
    models.append(model)
num = 0
for data in test_loader:
    ids = data['ids'].cuda()
    masks = data['masks'].cuda()
    tweet = data['tweet']
    offsets = data['offsets'].numpy()
    sentiment = data['sentiment']
    selected_text = data['selected_text']

    start_logits = []
    end_logits = []
    for model in models:
        with torch.no_grad():
            output = model(ids, masks)
            start_logits.append(torch.softmax(output[0], dim=1).cpu().detach().numpy())
            end_logits.append(torch.softmax(output[1], dim=1).cpu().detach().numpy())

    start_logits = np.mean(start_logits, axis=0)
    end_logits = np.mean(end_logits, axis=0)
    
    for i in range(len(ids)): 
        if sentiment[i] != 'neutral':
            start_pred = np.argmax(start_logits[i])
            end_pred = np.argmax(end_logits[i])
            if start_pred > end_pred:
                pred = tweet[i]
            else:
                pred = get_selected_text(tweet[i], start_pred, end_pred, offsets[i])
                if len(pred.split()) <= 1:
                    pred = pp(tweet[i], pred)
            predictions.append(pred)
        else:
            num += 1
            predictions.append(selected_text[i])
print(num)

# **Submission**

In [ ]:
# sub_df = pd.read_csv('../input/tweet-sentiment-extraction/sample_submission.csv')
# sub_df['selected_text'] = predictions
# sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('!!!!', '!') if len(x.split())==1 else x)
# sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('..', '.') if len(x.split())==1 else x)
# sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('...', '.') if len(x.split())==1 else x)
# sub_df.to_csv('submission.csv', index=False)
# sub_df.head()

test_df['selected_text'] = predictions
test_df[['textID','selected_text']].to_csv('submission.csv',index=False)
pd.set_option('max_colwidth', 60)
test_df.sample(25)



In [ ]:
# test_df = pd.read_csv("../input/tweet-sentiment-extraction/test.csv").set_index("textID")

# sub_df = pd.read_csv("../input/submission-2/submission (4).csv").set_index("textID")

# # Everything not presented in the public set 
# # will take a value of the original text
# test_df["selected_text"] = test_df.text

# # Get the public ids and assign them
# public_idxs = sub_df.index.values
# test_df.loc[public_idxs, "selected_text"] = sub_df.selected_text.values
# test_df[["selected_text"]].to_csv("submission.csv")